In [1]:
import numpy as np
import pandas as pd
import refesh_data as db

from datetime import datetime

In [43]:
def getNumCrowd():
    return db.dbNumCrowd()

db_store = getNumCrowd()
db_store.head(5)

,recordtime,in_num,out_num,position,storeid
0,2018-08-03 16:08:00,0,4,F02,29
1,2018-05-03 10:23:00,2,0,Floor 02,28
2,2018-05-03 10:25:00,1,1,Floor 02,28
3,2018-05-03 10:27:00,2,0,Floor 02,28
4,2018-05-03 10:29:00,1,0,Floor 02,28


In [ ]:
# Daily:   [y = None, d = '2023-03-10', w = None                 , m = None   , q = None]
# Weekly:  [y = 2023, d = None        , w = 'WK1 (01/01 - 01/01)', m = None   , q = None]
# Monthly: [y = 2023, d = None        , w = None                 , m = 'April', q = None]
# Quarter: [y = 2023, d = None        , w = None                 , m = None   , q = 0   ]
# Yearly:  [y = 2023, d = None        , w = None                 , m = None   , q = None]

# 2023-04-20 - <class 'datetime.date'>
# WK1 (01/01 - 01/01) - <class 'str'>
# April - <class 'str'>
# 0 - <class 'int'>
# 2023 - <class 'int'>

In [3]:
def filter_data(data, store = 0, d = None, w = None, m = None, q = None, y = None):
    if (not y) and d:
        # print('Daily')
        data = data[data.recordtime.dt.date == d]
    else:
        data = data[data.recordtime.dt.year == y]
        if w:
            print('Weekly')
            data = data[data.recordtime.dt.date >= w[0] & data.recordtime.dt.date <= w[1]]
        elif m:
            # print('Monthly')
            data = data[data.recordtime.dt.strftime('%B') == m]
        elif q:
            # print('Quarter')
            data = data[data.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int) == q + 1]

    if store > 0:
        data = data[data.storeid == store]

    return data

In [4]:
data = db_store.copy()

d = datetime.strptime('2023-03-17', '%Y-%m-%d').date()
df_daily = filter_data(data, 'All', d = d)
df_daily

# df_weekly = filter_data(data, 'All', w = 'WK1 (01/01 - 01/01)', y = 2023)
# df_weekly

# df_monthly = filter_data(data, 'All', m = 'April', y = 2023)
# df_monthly

# df_quarter = filter_data(data, 'All', q = 4, y = 2022)
# df_quarter

# df_yearly = filter_data(data, 'All', y = 2021)
# df_yearly

,recordtime,in_num,out_num,position,storeid
2917966,2023-03-17 10:24:00,56,38,Starbuck,34
2917967,2023-03-17 10:29:00,8,7,Starbuck,34
2917968,2023-03-17 10:34:00,5,13,Starbuck,34
2917969,2023-03-17 10:39:00,2,5,Starbuck,34
2917970,2023-03-17 10:44:00,16,0,Starbuck,34
...,...,...,...,...,...
2975790,2023-03-17 22:34:00,0,7,Floor 03,30
2975791,2023-03-17 22:39:00,2,16,Floor 03,30
2975792,2023-03-17 22:44:00,1,36,Floor 03,30
2975793,2023-03-17 22:49:00,2,18,Floor 03,30


In [ ]:
df = db_store.tail(205600).copy()
df['w'] = df['recordtime'].dt.strftime('%V').astype(int)
# df['s'] = df['w'].shift(1)
# df['week_num'] = np.where(df['s'].isna(), 1, df['w'] + 1)
# group = df.groupby('week_num').agg({'recordtime': ['min', 'max']}).reset_index()
# group['week'] = 'WK' + group['week_num'].astype(str) + \
#                 ' (' + group['recordtime']['min'].dt.strftime('%d/%m') + \
#                 ' - ' + group['recordtime']['max'].dt.strftime('%d/%m') + ')'

# df['week'] = df['recordtime'].dt.isocalendar().week

df.head()

In [44]:
def getWeekNums(year):
    start_date = '1/1/' + year
    end_date = '31/12/' + year
    data = pd.date_range(start = start_date, end = end_date, freq = 'D')

    df = pd.DataFrame(data, columns = ['date'])

    df['w'] = df['date'].dt.strftime('%V').astype(int)
    df['s'] = df['w'].shift(1)
    df['week_num'] = np.where(df['s'].isna(), 1, df['w'] + 1)
    group = df.groupby('week_num').agg({'date': ['min', 'max']}).reset_index()
    group['week'] = 'WK' + group['week_num'].astype(str) + \
                    ' (' + group['date']['min'].dt.strftime('%d/%m') + \
                    ' - ' + group['date']['max'].dt.strftime('%d/%m') + ')'

    return group

df = getWeekNums('2023')
df.head(2)

/var/folders/44/x_bsc3xn6vv4xqj9s0vfp7l80000gn/T/ipykernel_45304/2721257259.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data = pd.date_range(start = start_date, end = end_date, freq = 'D')


week_num       date                            week
                  min        max                     
0        1 2023-01-01 2023-01-01  WK1 (01/01 - 01/01)
1        2 2023-01-02 2023-01-08  WK2 (02/01 - 08/01)

In [54]:
w = 7
a = (df.loc(0)[w][1], df.loc(0)[w][2])
a

(Timestamp('2023-02-13 00:00:00'), Timestamp('2023-02-19 00:00:00'))